# Fixation calculation

In [3]:
import pandas as pd
import math
import ctypes
import numpy as np
from scipy.spatial.distance import pdist

The threshold within which the fixation points must be contained is 1º of visual angle, thus establishing this threshold for the implementation of our I-DT algorithm **[1]**. To translate this angle into distance in pixels, it is necessary to take into account the screen resolution and the inches of our device.

The ``minimum number of gaze points (N)`` for a fixation depends on the frequency h (in hertz) of the eyetracking
device. From the literature, fixation durations are typically estimated in the range of 60–400ms; in general, a minimum duration (dm) about 100 ms is a reasonable lower bound for information processing to occur. A straightforward choice of N is then **N = Hz(device) · dm/1,000 ms* **[1]**, **[2]**, **[3]**, **[4]**.

The screen resolution can be obtained using the native Python library "ctypes," but both the inches of the screen on which eye tracking will be performed and the distance from the observer to the screen must be adjusted manually. By default, and as done in laboratory tests, the established setup will be a 15.6" screen (standard in laptops) and a distance of 50 cm between the observer and the webcam device.

**[1]** ``Fixation identification: The optimum threshold for a dispersion algorithm. Blignaut (2009).``

**[2]** ``Inferring relevance from eye movements: Feature extraction. Saloj¨arvi J, Puolam¨aki K, Simola J, Kovanen L, Kojo I, Kaski S (2005). ``

**[3]** ``Identifying fixations and saccades ineye-tracking protocols. Salvucci DD, Goldberg JH (2000).``

**[4]** ``Standardization of automated analyses of oculomotor fixation and saccadic behaviors. Komogortsev OV,Gobert DV, Jayarathna S, Koh DH, Gowda SM (2010).``


**Identification-Dispersion Threshold (I-DT) Algorithm**

- Distance Threshold (pixels) and Fixation Minimum Duration (ms).

- Euclidean Distance within points (fixation). (I-DT)

**[3]** ``Identifying fixations and saccades ineye-tracking protocols. Salvucci DD, Goldberg JH (2000).``

The I-DT algorithm uses a moving window that spans consecutive data points to check for potential fixations. The moving window begins at the start of the protocol and initially spans a minimum number of points, determined by the given duration threshold and sampling frequency. I-DT then checks the dispersion of the points in the window by summing the differences between the points’ maximum and minimum x and y values; in other words, dispersion D = [max(x) – min(x)] + [max(y) – min(y)]. It is worth noting that alternative dispersion metrics could be based on spatial variance or the area of the samples. If the dispersion exceeds the dispersion threshold, the window does not represent a fixation, and the window moves one point to the right. If the dispersion is below the dispersion threshold, the window represents a fixation. In this case, the window is expanded (to the right) until the window’s dispersion exceeds the threshold. The final window is registered as a fixation at the centroid of the window points, with the given onset time and duration. This process continues with the window moving to the right until the end of the protocol is reached. Table 4 includes pseudocode for the I-DT algorithm.

We can note that this characterization of fixations uses the centroid and diameter. A circular area is usually assumed, and the mean distance from each sample to the fixation centroid provides an estimate of the radius. Additionally, dispersion-based algorithms are sometimes used to locate clusters within minimum spanning tree network representations (see the following section). The graph forms an efficient framework for rapid searches of large sample sets.

The I-DT algorithm requires two parameters: the dispersion threshold and the duration threshold. Like the velocity threshold for I-VT, the dispersion threshold can be set to include 1/2° to 1° of visual angle if the distance from the eye to the screen is known. Otherwise, the dispersion threshold can be estimated from exploratory analysis of the data. The duration threshold is typically set to a value between 100 and 200 ms [21], depending on task processing demands.

In [4]:
INCH_PER_CENTIMETRES = 2.54 #1 inch =  2.54cm
SCREEN_INCHES = 15.6 #Screen Inches
OBSERVER_CAMERA_DISTANCE = 50 #cm 
DEVICE_FREQUENCY = 15 #Hz. Frequency of our Eyetracker software (Webgazer)
FIXATION_MINIMUM_DURATION = 200 #ms (This is for Fixation Minimum Gazepoints, not I-VT)


def get_distance_threshold_by_resolution():
    user32 = ctypes.windll.user32
    user32.SetProcessDPIAware()
    width, height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1) #Screen Resolution
    
    angle_radians = np.radians(1)
    sin_value = np.sin(angle_radians)#Sin(1º) value
    print(f"sin(1º) = {sin_value}")
    
    radius_diameter = sin_value*50*2 #diámetro en cm
    print(f"Fixation Boundary (diameter): {radius_diameter} cm.")

    screen_diagonal_pixels = math.sqrt((width)**2 + (height)**2)#diagonal de la pantalla en píxeles. Dependiendo de la resolución de la pantalla, tendrá un valor diferente
    print(f"Screen Diagonal Resolution (in pixels): {screen_diagonal_pixels} px.")
    
    pixels_per_inches = screen_diagonal_pixels/SCREEN_INCHES
    print(f"Pixels per Inches: {pixels_per_inches} px/inches.")

    pixels_per_centimetres = pixels_per_inches/INCH_PER_CENTIMETRES
    print(f"Pixels per centimetres: {pixels_per_centimetres} px/centimetres.")

    pixels_threshold_i_dt = int(radius_diameter * pixels_per_centimetres)
    print(f"I-DT threshold (in pixels): {pixels_threshold_i_dt} px.")
    
    return pixels_threshold_i_dt

#pixels_threshold_i_dt  es la Window Size (I-DT) en píxeles

def get_minimum_fixation_gazepoints():
    
    minimum_fixation_gazepoints = round(DEVICE_FREQUENCY*FIXATION_MINIMUM_DURATION/1000)
    # print(f"The minimum gaze points considered to be a possible fixation is {minimum_fixation_gazepoints},\naccording to our device refresh rate and the established fixation minimum duration (100ms).\n")
    
    return minimum_fixation_gazepoints


In [5]:
#I-DT Algorithm

# def check_euclidean_distance(df, gaze_x_colname, gaze_y_colname, curr_df_index, cont, min_distance)
#     - df consulto todos los elementos de la columna gaze_x_colname desde el curr_df_index - cont hasta el curr_df_index en el df y los meto en una lista gaze_x_ls
#     - df consulto todos los elementos de la columna gaze_y_colname desde el curr_df_index - cont hasta el curr_df_index en el df y los meto en una lista gaze_y_ls
#     - calculo el maximo de gaze_x_ls y lo meto en una variable max_gaze_x
#     - calculo el maximo de gaze_y_ls y lo meto en una variable max_gaze_y
#     - calculo el minimo de gaze_x_ls y lo meto en una variable min_gaze_x
#     - calculo el minimo de gaze_y_ls y lo meto en una variable min_gaze_y
#     - calculo la raiz cuadrada de la suma de los cuadrados de (max_gaze_x - min_gaze_x) y de (max_gaze_y - min_gaze_y)
#     - y comparo esto con min_distance -> return del boolean resultante
    
def check_euclidean_distance(df, gaze_x_colname, gaze_y_colname, curr_df_index, cont, distance_threshold):
    gaze_x_ls = df[gaze_x_colname].iloc[curr_df_index - cont:curr_df_index + 1].tolist()
    gaze_y_ls = df[gaze_y_colname].iloc[curr_df_index - cont:curr_df_index + 1].tolist()

    max_gaze_x = max(gaze_x_ls)
    max_gaze_y = max(gaze_y_ls)
    min_gaze_x = min(gaze_x_ls)
    min_gaze_y = min(gaze_y_ls)

    distance = math.sqrt((max_gaze_x - min_gaze_x)**2 + (max_gaze_y - min_gaze_y)**2)

    return distance <= distance_threshold

# Ejemplo de uso:
# Supongamos que tienes un DataFrame 'df' con las columnas 'gaze_x' y 'gaze_y'
# y quieres verificar la distancia euclidiana con un índice actual de 5, un contador de 3 y una distancia mínima de 5.
# El resultado será True si la distancia es mayor o igual a 5, de lo contrario, será False.
# Puedes ajustar estos valores según tus necesidades.
# result = check_euclidean_distance(df, 'gaze_x', 'gaze_y', 5, 3, 5)
# print(result)


``` preprocess_gaze_log ``` es una función de preprocesamiento para los datos de seguimiento ocular (gaze data). Los datos de seguimiento ocular a menudo contienen puntos de fijación y sacadas. Las fijaciones son momentos en los que los ojos se quedan relativamente quietos, mientras que las sacadas son movimientos rápidos de los ojos entre las fijaciones. Esta función identifica las fijaciones y calcula varias estadísticas para cada una.

Aquí está lo que hace cada parte del código:

1. Inicializa varias variables para rastrear el estado actual de la fijación.

2. Recorre cada fila en el DataFrame de pandas `df`. Para cada fila, llama a la función `check_euclidean_distance` para determinar si el punto actual está lo suficientemente cerca del punto anterior para ser considerado parte de la misma fijación.

3. Si el punto actual está lo suficientemente cerca, incrementa el contador `cont` y calcula el centroide de la fijación actual.

4. Si el punto actual no está lo suficientemente cerca, reinicia el contador `cont`.

5. Si el contador `cont` es mayor o igual a `min_points`, entonces considera que la fijación es válida. Si es una nueva fijación, guarda la posición de inicio y fin de la última fijación y incrementa el índice de fijación.

6. Si `store_fixation` es verdadero, calcula varias estadísticas para la fijación actual, incluyendo el centroide, el inicio y el fin de la fijación, la duración de la fijación, la dispersión de la fijación, y almacena estos valores en nuevas columnas en `df`.

7. Finalmente, devuelve el DataFrame `df` con las nuevas columnas añadidas.

Por lo tanto, esta función es útil para preprocesar los datos de seguimiento ocular y calcular varias estadísticas para cada fijación.


La desviación estándar es una medida de la cantidad de variación o dispersión de un conjunto de valores. Una desviación estándar baja indica que los valores tienden a estar cerca de la media del conjunto, mientras que una desviación estándar alta indica que los valores están más dispersos.

Calcular el índice de dispersión como la media de las desviaciones estándar de x e y. Esto se denomina como coeficiente de variación. El coeficiente de variación es una medida de la dispersión relativa de forma porcentual.

In [6]:

def preprocess_gaze_log(df, gaze_x_colname, gaze_y_colname, min_points, distance_threshold):
    df = df.copy()
    df = df.rename_axis('RowNumber').reset_index()
    cont = 0 # curr_fixation_points que estan en el cluster que estoy estudiando
    pos_start_last_fixation = 0 # la posicion (row del df) del primer punto del fixation cluster
    pos_current_fixation = None
    fixation_index = 0 # el id que le voy a dar a este fixation cluster
    store_fixation = False
    for row_i, row in df.iterrows():
        aux = check_euclidean_distance(df, gaze_x_colname, gaze_y_colname, row_i, cont, distance_threshold)
        if aux:
            cont+=1 # hay un punto mas en el cluster
            centroid_x = df[gaze_x_colname].iloc[row_i - cont:row_i + 1].mean()
            centroid_y = df[gaze_y_colname].iloc[row_i - cont:row_i + 1].mean()
            
        else:
            cont=0 # se produce un saccade
        
        if cont >= min_points: 
            print("cont", cont)
            print("row_i", row_i)
            print("pos_current_fixation", pos_current_fixation)
            cond = pos_current_fixation and pos_current_fixation != row_i - 1 
            print("pos_current_fixation and pos_current_fixation != row_i - 1", cond)
            print("================================================================================")
            if cond: #Si es una fijación nueva... almaceno la posicion del cluster anterior
                pos_start_last_fixation_to_store = pos_start_last_fixation # se guarda la pos_start de la fijación
                pos_end_last_fixation = pos_current_fixation # la posición actual
                fixation_index +=1
                aux_pos = pos_start_last_fixation_to_store-1
                if pos_start_last_fixation_to_store-1 < 0:
                    aux_pos = 0
                    # last_fixation_start = df['Timestamp'].iloc[aux_pos:pos_start_last_fixation_to_store+1].mean()
                    # last_fixation_end = df['Timestamp'].iloc[pos_end_last_fixation:pos_end_last_fixation+2].mean()
                    # last_fixation_duration = last_fixation_end - last_fixation_start
                store_fixation = True
            pos_start_last_fixation = (row_i+1) - cont  #primer gaze_point del cluster
            pos_current_fixation = row_i
            
        if store_fixation:            
            if cont > 0:
                # Calcular el valor medio o centroide de 'gaze_x_colname' y 'gaze_y_colname' para el cluster de fijación actual
                centroid_x = df[gaze_x_colname].iloc[pos_start_last_fixation_to_store:pos_end_last_fixation + 1].mean()
                centroid_y = df[gaze_y_colname].iloc[pos_start_last_fixation_to_store:pos_end_last_fixation + 1].mean()
                
                # Calcular el comienzo de una fijación (tiempo medio entre el ultimo punto que es una sacada 
                # y el primer punto que es una fijación) 
                last_fixation_start = df['Timestamp'].iloc[aux_pos:pos_start_last_fixation_to_store+1].mean()
                # y el final de una fijación (tiempo medio entre el ultimo punto que es una fijación 
                # y el primer punto que es una sacada)
                last_fixation_end = df['Timestamp'].iloc[pos_end_last_fixation:pos_end_last_fixation+2].mean()
                # y la duración de la fijación (tiempo entre el comienzo y el final de la fijación)
                last_fixation_duration = last_fixation_end - last_fixation_start
                
                # Calcular la desviación estándar de las coordenadas x e y de los puntos de fijación
                #La desviación estándar es una medida de la cantidad de variación o dispersión
                # de un conjunto de valores. Una desviación estándar baja indica que los valores tienden a estar cerca de la media del conjunto,
                # mientras que una desviación estándar alta indica que los valores están más dispersos.
                
                fixation_dispersion_x = df[gaze_x_colname].iloc[pos_start_last_fixation_to_store:pos_end_last_fixation + 1].std()
                fixation_dispersion_y = df[gaze_y_colname].iloc[pos_start_last_fixation_to_store:pos_end_last_fixation + 1].std()
                
                # Calcular el índice de dispersión como la media de las desviaciones estándar de x e y. Esto se denomina
                #como coeficiente de variación. El coeficiente de variación es una medida de la dispersión relativa de forma 
                #porcentual
                fixation_dispersion = round(((fixation_dispersion_x + fixation_dispersion_y) / 2) / 100, 2)
                

            else:
                centroid_x = 0
                centroid_y = 0
                fixation_dispersion_x = 0
                fixation_dispersion_y = 0
                fixation_dispersion = 0


            # añadir el fixation index desde la row pos_start_last_fixation hasta el pos_current_fixation en la columna de fixation_index del df
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation Index'] = fixation_index
            # Almacenar el índice de fijación y el valor medio en nuevas columnas
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation X'] = centroid_x
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation Y'] = centroid_y
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation Start'] = last_fixation_start
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation End'] = last_fixation_end
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation Duration'] = last_fixation_duration
            # Indice de dispersión: Puede calcularse como la relación entre la media de las distancias entre todos los puntos 
            # y la distancia media desde cada punto hasta el centro del conjunto. 
            # Un índice mayor indica mayor dispersión.
            df.loc[pos_start_last_fixation_to_store:pos_end_last_fixation, 'Fixation Dispersion'] = fixation_dispersion
            
    
            
            # print("estoy guardando los fixation")
            # print("fixation_index: ", fixation_index)
            # print("pos_start_last_fixation: ", pos_start_last_fixation)
            # print("pos_end_last_fixation: ", pos_end_last_fixation)
            # print("cont: ", cont)
            store_fixation = False
            
    return df

In [40]:
df = pd.read_csv("webgazer_gazeData.csv")

In [41]:
df.head()

,Gaze X,Gaze Y,Timestamp
0,500.51,499.92,1
1,500.12,498.89,69
2,499.79,491.03,132
3,510.02,465.84,194
4,536.77,426.28,256


In [42]:
preprocess_df = preprocess_gaze_log(df, "Gaze X", "Gaze Y", get_minimum_fixation_gazepoints(), get_distance_threshold_by_resolution())
print("fixation min points for this experiment", get_minimum_fixation_gazepoints())

sin(1º) = 0.01745240643728351
Fixation Boundary (diameter): 1.7452406437283512 cm.
Screen Diagonal Resolution (in pixels): 2202.9071700822983 px.
Pixels per Inches: 141.21199808219862 px/inches.
Pixels per centimetres: 55.59527483551127 px/centimetres.
I-DT threshold (in pixels): 97 px.
cont 2
row_i 1
pos_current_fixation None
pos_current_fixation and pos_current_fixation != row_i - 1 None
cont 3
row_i 2
pos_current_fixation 1
pos_current_fixation and pos_current_fixation != row_i - 1 False
cont 4
row_i 3
pos_current_fixation 2
pos_current_fixation and pos_current_fixation != row_i - 1 False
cont 5
row_i 4
pos_current_fixation 3
pos_current_fixation and pos_current_fixation != row_i - 1 False
cont 2
row_i 9
pos_current_fixation 4
pos_current_fixation and pos_current_fixation != row_i - 1 True
cont 2
row_i 12
pos_current_fixation 9
pos_current_fixation and pos_current_fixation != row_i - 1 True
cont 3
row_i 13
pos_current_fixation 12
pos_current_fixation and pos_current_fixation != row_

In [43]:
preprocess_df


,RowNumber,Gaze X,Gaze Y,Timestamp,Fixation Index,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion
0,0,500.51,499.92,1,1.0,509.442,476.392,1.0,287.5,286.5,0.24
1,1,500.12,498.89,69,1.0,509.442,476.392,1.0,287.5,286.5,0.24
2,2,499.79,491.03,132,1.0,509.442,476.392,1.0,287.5,286.5,0.24
3,3,510.02,465.84,194,1.0,509.442,476.392,1.0,287.5,286.5,0.24
4,4,536.77,426.28,256,1.0,509.442,476.392,1.0,287.5,286.5,0.24
...,...,...,...,...,...,...,...,...,...,...,...
372,372,1551.08,7.08,25875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,373,1551.46,7.07,25941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,374,1574.41,-12.79,26003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,375,1595.06,-11.15,26067,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Procesamientos del gaze_log para que ScreenRPA pueda realizar el mapping entre este GazeLog y el UI Log (.mht)
1º Se añade un index a aquellos puntos considerados sacádicos (no pertenecen a grupos de fixations)
2º Se transforma Fixation Index y Saccade Index a int64.

In [44]:
#AJUSTES PARRA SCREENRPA

def add_saccade_index(df):
    df = df.copy()
    df['Saccade Index'] = None
    saccade_index = 1
    for row_i, row in df.iterrows():
        if pd.isnull(row['Fixation Index']):
            df.loc[row_i, 'Saccade Index'] = saccade_index
            saccade_index += 1
    return df

preProcessed_saccade_index_df = add_saccade_index(preprocess_df)

def int_index(df):
    df = df.copy()
    df['Saccade Index'] = df['Saccade Index'].astype('Int64')
    df['Fixation Index'] = df['Fixation Index'].astype('Int64')
    return df

postprocessed_df = int_index(preProcessed_saccade_index_df)



In [45]:
postprocessed_df.to_csv("webgazer_gazeData_postProcessed.csv")
postprocessed_df



,RowNumber,Gaze X,Gaze Y,Timestamp,Fixation Index,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Saccade Index
0,0,500.51,499.92,1,1,509.442,476.392,1.0,287.5,286.5,0.24,<NA>
1,1,500.12,498.89,69,1,509.442,476.392,1.0,287.5,286.5,0.24,<NA>
2,2,499.79,491.03,132,1,509.442,476.392,1.0,287.5,286.5,0.24,<NA>
3,3,510.02,465.84,194,1,509.442,476.392,1.0,287.5,286.5,0.24,<NA>
4,4,536.77,426.28,256,1,509.442,476.392,1.0,287.5,286.5,0.24,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
372,372,1551.08,7.08,25875,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,95
373,373,1551.46,7.07,25941,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,96
374,374,1574.41,-12.79,26003,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,97
375,375,1595.06,-11.15,26067,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,98


In [47]:

print(postprocessed_df.dtypes)


RowNumber                int64
Gaze X                 float64
Gaze Y                 float64
Timestamp                int64
Fixation Index           Int64
Fixation X             float64
Fixation Y             float64
Fixation Start         float64
Fixation End           float64
Fixation Duration      float64
Fixation Dispersion    float64
Saccade Index            Int64
dtype: object
